# 새 데이터 가져오기

# eda
## 사명 변경된 적 있는지
```
final_df = df[df.groupby('company_sn')['company_name'].transform('nunique') > 1].sort_values(by='job_idx')[['company_sn', 'company_name']].drop_duplicates()
final_df['name_count'] = final_df.groupby('company_sn').cumcount() + 1
final_df.pivot_table(columns='name_count', index='company_sn', values='company_name', aggfunc='first')
```

## 목록에 표시된 해시태그
- 하지만 각 공고마다 가지고 있는 태그 전체 내용이 아니다.
```
df2 = df[['job_idx','job_sectors']]
df2.explode('job_sectors').groupby('job_sectors').size().sort_values(ascending=False).to_csv('job_sectors.csv')
df.explode('job_sectors').groupby('job_sectors').get_group('크롤링')

## jobsector 데이터가 조회할때마다 달라지는지 확인 필요
- 다섯개 이상 선택해도 리스트에 표시되지 않음
- 지정된 우선순위 순서로 표시되는지 아니면 조회할때마다 무작위로 5개 선택하는지 확인 필요

# 구글 시트에서 이전 시트 가져오기

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

gc = gspread.service_account()

sh=gc.open_by_key('1vkZ7FF1bgACwXKxkmSNemmSNiti4Bd2M177w-8H80ww').sheet1




# modified          datetime64[ns]
# resistered        datetime64[ns]
# update            datetime64[ns]
# df_old의 세가지 열 데이터타입 변경


df_old = get_as_dataframe(sh, evaluate_formulas=False)
df_old.dropna(subset=['job_idx'], inplace=True)
df_old['job_idx'] = df_old['job_idx'].astype(int)
df_old.set_index('job_idx', inplace=True)
df_old['is_headhunting'] = df_old['is_headhunting'].astype(bool)
df_old["modified"] = pd.to_datetime(df_old["modified"])
df_old["resistered"] = pd.to_datetime(df_old["resistered"])
df_old["update"] = pd.to_datetime(df_old["update"])
df_old

# 새시트, 이전시트 병합
- jobsector 열 리스트 확장
- 제목 변경되는 경우 특이 케이스로 보고 따로 표시
- 수정일 : 새 시트 우선
- 작성일 : 이전 시트 우선, 원래 없다가 새로 생겨나는 경우 없을듯
- update 열 : 마지막으로 조회한 시간, 새 시트 우선
- 어떻게 합치는게 세련된 방법인지 모르겠다

- old에서 안쓰는 열은 드롭? 안돼 그러면 아우터가 아니니까

- 작성일 관측 불가면 어떻게 할지?

# Merge the dataframes
merged_df = pd.merge(df_old, df_new, how='outer', on=['job_idx'])

# Apply conditions to overlapping parts
# modified new 우선, resistered old 우선, update new 우선

merged_df["resistered"] = merged_df["resistered_x"].fillna(merged_df["resistered_y"])
merged_df["modified"] = merged_df["modified_x"].fillna(merged_df["modified_y"])
merged_df["update"] = merged_df["update_y"].fillna(merged_df["update_x"])

# Drop the columns
merged_df = merged_df.drop(columns=["resistered_x", "resistered_y", "modified_x", "modified_y", "update_x", "update_y"])
merged_df

# 새 시트로 덮어쓰기

# Clear the contents of the sheet
sh.clear()

# Set the new dataframe in the sheet
set_with_dataframe(sh, combined_df, include_index=True, allow_formulas=False)


# 키워드 처리
- job_sector 열에 키워드 들어있다.
- 키워드는 리스트 타입으로
- 각 공고마다 5개 이상의 키워드 가질 수 있다.
- 공고 목록에 표시되는 건 5개의 키워드 뿐
- 키워드로 목록을 새로 조회할 수 있다.
- 키워드로 목록을 조회할 때, 해당 목록의 모든 공고가 그 키워드를 가지고 있다고 등록해줘야함
- 자동차 라는 키워드로 목록 조회
- 그 목록을 1차 처리
- 키워드 리스트를 별도 테이블로 분리하고, 모든 공고에 검색 키워드 추가


전산총무 : 415
자동차 : 2218


In [2]:
import pandas as pd
import pickle

In [3]:
# Specify the file path of the pickle file
file_path = 'df_key.pickle'

# Load the pickle file
with open(file_path, 'rb') as file:
    df_key = pickle.load(file)


In [4]:
# Specify the file path of the pickle file
file_path = 'df_combined.pickle'

# Load the pickle file into df_old
with open(file_path, 'rb') as file:
    df_job = pickle.load(file)


In [5]:
df_job.shape

(4911, 14)

In [7]:
df_key.groupby('job_idx').size().sort_values(ascending=False)


job_idx
47905963    8
47871493    8
47862010    8
47896473    8
47889275    8
           ..
47911923    1
47836917    1
47910300    1
47783072    1
47838204    1
Length: 4911, dtype: int64